In [ ]:
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

In [ ]:
import networkx as nx
import math
from PIL import Image, ImageDraw
import pyproj
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
import pandas as pd

In [ ]:
G = nx.DiGraph()

In [ ]:
airportsDF = pd.read_csv('airports.dat', header=None, names=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

for index, row in airportsDF.iterrows():
    if row['IATA'] == '\\N':
        #print(row['IATA'])
        continue
    G.add_node(row['IATA'], pos=(row['lat'],row['lon']))

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
print(G.nodes['TRF'])
#print(G.nodes['AOS'])

In [ ]:
routesDF = pd.read_csv('routes.dat', header=None, names=['airline', 'airlineID', 'source', 'sourceID', 'dest', 'destID', 'codeshare', 'stops', 'equipment'])

for index, row in routesDF.iterrows():
    if(row['source'] not in G.nodes or row['dest'] not in G.nodes):
        continue
    dist = geopy.distance.distance((G.nodes[row['source']]['pos']), (G.nodes[row['dest']]['pos'])).km
    G.add_edge(row['source'], row['dest'], weight=dist)

In [ ]:
print(len(G.nodes))

In [ ]:
# remove nodes with zero degree
#G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
G.is_directed()

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))
H = G.copy()
i=0
while i == 0:
  i = 1
  H = G.copy()
  for node in H.in_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])
  H = G.copy()
  for node in H.out_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)
print(G.degree)

In [ ]:
print(len(G.nodes))

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1])
print(nodes)

In [ ]:
print(len(G.nodes))
print(len(G.edges))

In [ ]:
print(G.degree("LIS"))
print(G.degree("ZYL"))
print(G.degree("AAE"))

In [ ]:
# Graph density
nx.density(G)

In [ ]:
# Graph diameter
nx.diameter(G)

In [ ]:
#TODO logaritmic scale
# Degree Distribution
degrees = [G.degree(n) for n in G.nodes()]
print(max(degrees))
plt.hist(degrees, bins=int(max(degrees)/10))
# for i in range(0, max(degrees)):
#   print(i, degrees.count(i))

In [ ]:
# compute graph degree centrality
dc = nx.degree_centrality(G)
print(max(dc, key=dc.get))
print(min(dc, key=dc.get))

In [ ]:
# compute eigen vector centrality
ec = nx.eigenvector_centrality(G)
print(max(ec, key=ec.get))
print(min(ec, key=ec.get))

In [ ]:
# compute closeness centrality
cc = nx.closeness_centrality(G)
print(max(cc, key=cc.get))
print(min(cc, key=cc.get))

In [ ]:
# compute betweenness centrality
bc = nx.betweenness_centrality(G)
print(max(bc, key=bc.get))
print(min(bc, key=bc.get))

In [91]:
#FIXME erro
# compute pagerank
pr = nx.pagerank(G)
print(max(pr))
print(min(pr))

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

In [92]:
# compute clustering coefficient
cl = nx.clustering(G)
print(cl)

{'GKA': 1.0, 'MAG': 0.5714285714285714, 'HGU': 0.390625, 'LAE': 0.3888888888888889, 'POM': 0.11553030303030302, 'WWK': 0.8333333333333334, 'UAK': 0.3, 'GOH': 0.17857142857142858, 'SFJ': 0.2857142857142857, 'THU': 0, 'AEY': 0, 'EGS': 0, 'IFJ': 0, 'KEF': 0.4926644581164221, 'RKV': 0, 'YAM': 0.75, 'YAY': 0.6666666666666666, 'YBC': 0.9166666666666666, 'YBG': 0.74, 'YBK': 0.5555555555555556, 'YBL': 1.0, 'YBR': 0, 'YCB': 0.6666666666666666, 'YCD': 1.0, 'YCG': 1.0, 'YCL': 0, 'YCO': 0.3333333333333333, 'YCY': 0, 'YZS': 0.3333333333333333, 'YDF': 0.65625, 'YDQ': 0, 'YEG': 0.3239590942293645, 'YEK': 0.6666666666666666, 'YEV': 0.09523809523809523, 'YFB': 0.022222222222222223, 'YFC': 1.0, 'YFO': 1.0, 'YFS': 0, 'YGK': 0, 'YGL': 0.3333333333333333, 'YGP': 1.0, 'YGR': 0.5, 'YHD': 0.3333333333333333, 'YHI': 0.3333333333333333, 'YHK': 1.0, 'YHM': 0.5, 'YHU': 0.4166666666666667, 'YHY': 1.0, 'YHZ': 0.2802441731409545, 'YIO': 0, 'YJT': 1.0, 'YKA': 1.0, 'YKF': 1.0, 'YKL': 0.4166666666666667, 'YLL': 0, 'YLW

In [ ]:
print(G.degree["ZYL"])

In [ ]:
# generate strongly connected components
scc = nx.strongly_connected_components(G)
largest = max(scc, key=len)
print(len(largest))
#for node in largest:
  #print(G.nodes[node].get('country'))

In [ ]:
# average shortest path
avg_path_length = nx.average_shortest_path_length(G)
print(avg_path_length)


In [ ]:
G.nodes['LIS']

In [ ]:
# Draw the graph in a map
# latitude = float(G.nodes['LIS'].get('pos')[0])
# longitude = float(G.nodes['LIS'].get('pos')[1])

# #resolution of image you are using
# mapWidth = 2048;
# mapHeight = 1025;

# #get x value
# x = (mapWidth*(180+longitude)/360)%mapWidth+(mapWidth/2);
# #convert from degrees to radians
# latRad = latitude*math.pi/180;
# #get y value
# mercN = math.log(math.tan((math.pi/4)+(latRad/2)));
# y = (mapHeight/2)-(mapWidth*mercN/(2*math.pi));
# print(x, y)

# Define the source and destination projections
source_projection = pyproj.Proj(init='epsg:4326')  # WGS 84 (standard lat/lon)
destination_projection = pyproj.Proj(init='epsg:3857')  # Web Mercator

# Input geographic coordinates (longitude, latitude)
lon, lat = float(G.nodes['LIS'].get('pos')[0]), float(G.nodes['LIS'].get('pos')[1])  # Example coordinates for New York City

# Project the coordinates to the destination projection
x, y = pyproj.transform(source_projection, destination_projection, lon, lat)
print(x, y)

def project_to_pixel(x, y, scale, x_offset, y_offset):
    pixel_x = (x - x_offset) * scale
    pixel_y = (y - y_offset) * scale
    return pixel_x, pixel_y

# Calculate the pixel coordinates
scale = 0.00001 # Adjust as needed
x_offset, y_offset = 0, 0  # Adjust as needed
x, y = project_to_pixel(x, y, scale, x_offset, y_offset)
print(x, y)
image = Image.open("openflights-apdb-2048.png")
draw = ImageDraw.Draw(image)
draw.ellipse((x,y, x+50, y+50), fill="black")
image.show()